In [1]:
# dependencies
import json
import requests
import pandas as pd
from pprint import pprint
# need to pip install world bank data to access more detailed info
# pip install -U wbdata
import wbdata

In [2]:
# owid json on GitHub url
owid_covid19_url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.json"

In [3]:
# owid json json requests
owid_vaccinations_response = requests.get(owid_covid19_url).json()

In [4]:
### owid json on GitHub of vaccinations - API call and DataFrame creation

json_dict = {
    "country": [],
    "iso_code": [],
    "date": [],
    "daily_vaccinations": [],
    "daily_vaccinations_per_million": []
}

for index, x in enumerate(owid_vaccinations_response):
    results = owid_vaccinations_response[index]

    for index, data in enumerate(results["data"]):
        json_dict["country"].append(results["country"])
        json_dict["iso_code"].append(results["iso_code"])
            
        json_dict["date"].append(results["data"][index]["date"])
            
        try:
            json_dict["daily_vaccinations"].append(results["data"][index]["daily_vaccinations"])
        except(KeyError):
            json_dict["daily_vaccinations"].append("No Data")
                
        try:    
            json_dict["daily_vaccinations_per_million"].append(results["data"][index]["daily_vaccinations_per_million"])
        except(KeyError):
            json_dict["daily_vaccinations_per_million"].append("No Data")
            

raw_owid_vaccinations_data = pd.DataFrame(json_dict)

april_owid_vaccinations_data = raw_owid_vaccinations_data.loc[raw_owid_vaccinations_data["date"].str.contains("2021-04"), :]
april_owid_vaccinations_data.head()

,country,iso_code,date,daily_vaccinations,daily_vaccinations_per_million
38,Afghanistan,AFG,2021-04-01,3000,77
39,Afghanistan,AFG,2021-04-02,3000,77
40,Afghanistan,AFG,2021-04-03,3000,77
41,Afghanistan,AFG,2021-04-04,3000,77
42,Afghanistan,AFG,2021-04-05,3000,77


In [6]:
# testing wbdata install
#wbdata.get_source()
# finding available indicators to refine data
#wbdata.search_indicators('gdp per capita')
# confirming country list
#wbdata.get_country()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# iterating through countries to get their Gross Domestic Product per Capita
countries = [i['name'] for i in wbdata.get_country()]
indicators = { "NY.GDP.PCAP.CD" : "gdppc"}

# Sorting new data into a dataframe
wbdata_df = wbdata.get_dataframe(indicators, country='all')
wbdata_df = wbdata_df.reset_index()
wbdata_2019_df = wbdata_df[wbdata_df.date == "2019"]

#import pdb; pdb.set_trace()

clean_wbdata_2019_df = wbdata_2019_df


# get rid of null columns

clean_wbdata_2019_df = clean_wbdata_2019_df.dropna()
clean_wbdata_2019_df = clean_wbdata_2019_df.reset_index(drop=True)

# Verifying dataframe is how we want it

format_dict = {'gdppc':'${0:,.2f}'}
clean_wbdata_2019_df.style.format(format_dict).hide_index()

country,date,gdppc
Arab World,2019,"$6,584.74"
Caribbean small states,2019,"$10,500.97"
Central Europe and the Baltics,2019,"$16,298.48"
Early-demographic dividend,2019,"$3,644.01"
East Asia & Pacific,2019,"$11,502.87"
East Asia & Pacific (excluding high income),2019,"$8,192.14"
East Asia & Pacific (IDA & IBRD countries),2019,"$8,278.07"
Euro area,2019,"$38,976.34"
Europe & Central Asia,2019,"$24,742.72"
Europe & Central Asia (excluding high income),2019,"$8,133.57"
